In [36]:
import numpy
import scipy.linalg as sl
from numba import jit

In [37]:
#Input para el programa

N = 12
L = 5
X = numpy.array([0, 2, 4, 6, 6, 6, 6, 4, 2, 0, 0, 0, 0])
Y = numpy.array([0, 0, 0, 0, 2, 4, 6, 6, 6, 6, 4, 2, 0])
KODE = numpy.array([1, 1, 1, 0, 0 ,0, 1, 1, 1, 0, 0, 0])
FI = numpy.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 300, 300, 300])
CX = numpy.array([2, 2, 3, 4, 4])
CY = numpy.array([2, 4, 3, 2, 4])

In [76]:
# Simil de la función GHMATPC

def GHMATPC(X, Y, FI, KODE, NX):
    G = numpy.zeros((NX, NX))
    H = numpy.zeros((NX, NX))
    XM = numpy.zeros(NX)
    YM = numpy.zeros(NX)
    DFI = numpy.empty_like(FI)
    
    XM[-1] = (X[-1] + X[-2])/2
    YM[-1] = (Y[-1] + Y[-2])/2
    XM[:-1] = (X[1:-1] + X[:-2])/2
    YM[:-1] = (Y[1:-1] + Y[:-2])/2
    
    for i in range(N):
        for j in range(N):
            if i-j < 0 or i-j > 0:
                G[i,j] = EXTINPC(XM[i], YM[i], X[j], Y[j], X[j+1], Y[j+1], H[i,j], G[i,j], 0, N)[4]
            elif i-j == 0:
                G[i,j] = LOCINPC(X[j], Y[j], X[j+1], Y[j+1], G[i,j])
                H[i,j] = numpy.pi
                
    for i in range(N):
        if KODE[i] > 0:
            for j in range(N):
                CH = G[i,j]
                G[i,j] = -H[i,j]
                H[i,j] = -CH
    for i in range(N):
        DFI[i] = 0
        for j in range(N):
            DFI[i] = DFI[i] + H[i,j]*FI[j]
    return G, DFI, XM, YM

In [72]:
def EXTINPC(XP, YP, X1, Y1, X2, Y2, H, G, K, NX):
    GI = numpy.array([0.86113631, -0.86113631, 0.33998104, -0.33998104])
    OME = numpy.array([0.34785485, 0.34785485, 0.65214515, 0.65214515])
    XCO = numpy.zeros(NX)
    YCO = numpy.zeros(NX)
    
    AX = (X2 - X1)/2
    BX = (X2 + X1)/2
    AY = (Y2 - Y1)/2
    BY = (Y2 + Y1)/2
    SL = numpy.sqrt(AX**2 + AY**2)
    ETA_1 = AY/SL
    ETA_2 = -AX/SL
    G = 0
    H = 0
    DU1 = 0
    DU2 = 0
    DQ1 = 0
    DQ2 = 0
    
    for i in range(4):
        XCO[i] = AX*GI[i] + BX
        YCO[i] = AY*GI[i] + BY
        RA = numpy.sqrt((XP - XCO[i])**2 + (YP - YCO[i])**2)
        RD1 = (XCO[i] - XP)/RA
        RD2 = (YCO[i] - YP)/RA
        RDN = RD1*ETA_1 + RD2*ETA_2
        if K <= 0:
            G = G + numpy.log(1/RA)*OME[i]*SL
        else:
            DU1 = DU1 + RD1*OME[i]*SL/RA
            
        DU2 = DU2 + RD2*OME[i]*SL/RA
        DQ1 = DQ1 - ((2*RD1**2 - 1)*ETA_1 + 2*RD1*RD2*ETA_2)*OME[i]*SL/RA**2
        DQ2 = DQ2 - ((2*RD2**2 - 1)*ETA_2 + 2*RD1*RD2*ETA_1)*OME[i]*SL/RA**2
        H = H - RDN*OME[i]*SL/RA
    
    return DU1, DU2, DQ1, DQ2, G, H

In [50]:
def LOCINPC(X1, Y1, X2, Y2, G):
    AX = (X2 - X1)/2
    AY = (Y2 - Y1)/2
    SR = numpy.sqrt(AX**2 + AY**2)
    G = 2*SR*(1 - numpy.log(SR))
    return G

In [41]:
def SLNPD(A, DFI):
    MAT_F = sl.solve(A, DFI)
    return MAT_F

In [56]:
def INTERPC(FI, DFI, KODE, CX, CY, X, Y, N):
    for i in range(N):
        if KODE[i] > 0:
            dummy = FI[i]
            FI[i] = DFI[i]
            DFI[i] = dummy
    
    for i in range(L):
        POT[i] = 0
        FLUX1[i] = 0
        FLUX2[i] = 0
        for j in range(N):
            EXTINPC(CX[i], CY[i], X[i], Y[i], X[i+1], Y[i+1], A, B, 1, N)
            POT[i] = POT[i] + DFI[j]*B - FI[j]*A
            FLUX1[i] = FLUX1[i] + DFI[j]*DU1 - FI[j]*DQ1
            FLUX2[i] = FLUX2[i] + DFI[j]*DU2 - FI[j]*DQ2
        POT[i] = POT[i]/(2*numpy.pi)
        FLUX1[i] = FLUX1[i]/(2*numpy.pi)
        FLUX2[i] = FLUX2[i]/(2*numpy.pi)
    return POT, FLUX1, FLUX2  



In [80]:
Mat_A, Mat_DFI = GHMATPC(X, Y, FI, KODE, N)[0:2]

In [81]:
SLNPD(Mat_A, Mat_DFI)

array([ -594.92117728,  -834.29021169, -1022.41135442,  1341.28446412,
         491.26723344,  1341.28446412, -1022.41135442,  -834.29021169,
        -594.92117728,  1260.94148539,   546.2927141 ,  1260.94148539])